<a href="https://colab.research.google.com/github/marco-siino/eloquent2024/blob/main/ELOQUENT_2024_Task_2_Hallucigen_Mistral7B_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [ ]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets
!pip install -U deep-translator

import datasets
from datasets import load_dataset
from deep_translator import GoogleTranslator
import json
import re
import random
import numpy as np
import tqdm.notebook as tqdm

# Seed to shuffle the json training set.
seed_value = 42
random.seed(seed_value)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 MB 206.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 320.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 234.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 329.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.71-cp310-cp310-linux_x86_64.whl size=55332522 sha256=15b2e8a86ee262e2d37ec3c6536840a87ce93f17251b39c5bd215f1cfa7dd5b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-zeokkhh8/wheels/5d/35/d3/b4ab42b4528cc5e1f9b1469c4b4076de51e2f3ceb50506b424
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
 

Downloading an instruction-finetuned Mistral model.

In [ ]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loade

# Download the dataset for the three subtasks.

In [ ]:
# Subtask 1.
!wget 'https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2023_test_clean.json'
!wget 'https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2024_training.json'
!wget 'https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2024_training_task1_gold_hard.json'


train_path = 'EXIST2024_training.json'
train_gold_path = 'EXIST2024_training_task1_gold_hard.json'
test_path = 'EXIST2023_test_clean.json'

--2024-05-09 13:46:46--  https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2023_test_clean.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2143912 (2.0M) [text/plain]
Saving to: ‘EXIST2023_test_clean.json’

EXIST2023_test_clea 100%[===================>]   2.04M  --.-KB/s    in 0.008s  

2024-05-09 13:46:47 (245 MB/s) - ‘EXIST2023_test_clean.json’ saved [2143912/2143912]

--2024-05-09 13:46:47--  https://raw.githubusercontent.com/marco-siino/EXIST2024/main/dataset/EXIST2024_training.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awa

# Create few-shot samples from training set.

In [ ]:
 prompt_context = "[INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.\n"


In [ ]:
with open(train_path, 'r') as istr:
    train_1_json = json.load(istr)
  #print(train_1_json[0])

# Convert the dictionary keys into a list
keys_list = list(train_1_json.keys())

# Shuffle the list
random.shuffle(keys_list)

# Create a new dictionary with shuffled keys
train_1_json = {key: train_1_json[key] for key in keys_list}

In [ ]:
def create_few_shot_samples_run1(train_1_json,nr_samples):

  nr_few_shot_samples = nr_samples

  few_shot_counter = 0

  few_shot_samples = ''

  with open(train_gold_path, 'r') as istr:
    train_gold_json = json.load(istr)
  indexed_data = {entry["id"]: entry for entry in train_gold_json}

  for key, value in train_1_json.items():

    if few_shot_counter == nr_few_shot_samples:
      break
    try:
      label = indexed_data[key]['value']
    except:
      continue

    few_shot_counter+=1
    tweet = train_1_json[key]['tweet']
    print(train_1_json[key]['tweet'] + ' // ' + label)
    if train_1_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(train_1_json[key]['tweet'])

    few_shot_samples += '<s>'+prompt_context+"TWEET:"+tweet + ' [/INST] \n ' + label + "\n</s> \n\n "

    if few_shot_counter == nr_few_shot_samples:
        break

  return few_shot_samples

In [ ]:
def create_few_shot_samples_run2(train_1_json,nr_samples):

  nr_few_shot_samples = nr_samples

  few_shot_counter = 0

  few_shot_samples = ''

  with open(train_gold_path, 'r') as istr:
    train_gold_json = json.load(istr)
  indexed_data = {entry["id"]: entry for entry in train_gold_json}
  #print(indexed_data['100001'])

  for key, value in train_1_json.items():

    if few_shot_counter == nr_few_shot_samples:
      break
    try:
      label = indexed_data[key]['value']
    except:
      continue

    few_shot_counter+=1
    tweet = train_1_json[key]['tweet']
    print(train_1_json[key]['tweet'] + ' // ' + label)
    if train_1_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(train_1_json[key]['tweet'])

    few_shot_samples += prompt_context+"TWEET:"+tweet + ' [/INST] \n ' + label + "\n \n\n "

    if few_shot_counter == nr_few_shot_samples:
        break

  return few_shot_samples

In [ ]:
few_shot_samples_run1 = create_few_shot_samples_run1(train_1_json,10)

@JackelineWB @JohnMAckerman @lopezobrador_ tu eres el absurdo, o sea Loret si tiene derecho de violar la privacidad de personas civiles?? // NO
Yo me pregunto si que haya fortunas genera acaso un techo de cristal o algo por el estilo. Cada vez hay más ricos, y a quien le moleste lo heredado, que analice si padece de envidia. https://t.co/oesBZ03sdh // NO
No te vistas así*No uses minifalta*No se saques selfie*No salgas sola* Parecés una puta*Vos lo habrás mirado*Estás provocandoAh ! y que deprimente lo de Fablet. El señor que nos dice qué tenemos que hacer para que no nos violen. // YES
@Russtophocles1 Girls don’t count as reply guys , call me sexist but I always love compliments from da gals. Even if they are horny posting😂😂 but if you’re mean I will cry // YES
Siempre he pensado esto de todos los abogados ... Jajaja Bueno mi cuñado NO es así, me sorprende porque es el estereotipo perfecto! Costeño, ebrio y abogado https://t.co/YH6AjDlElH // NO
Es algo relacionado con la identidad de g

In [ ]:
few_shot_samples_run2 = create_few_shot_samples_run2(train_1_json,10)

@JackelineWB @JohnMAckerman @lopezobrador_ tu eres el absurdo, o sea Loret si tiene derecho de violar la privacidad de personas civiles?? // NO
Yo me pregunto si que haya fortunas genera acaso un techo de cristal o algo por el estilo. Cada vez hay más ricos, y a quien le moleste lo heredado, que analice si padece de envidia. https://t.co/oesBZ03sdh // NO
No te vistas así*No uses minifalta*No se saques selfie*No salgas sola* Parecés una puta*Vos lo habrás mirado*Estás provocandoAh ! y que deprimente lo de Fablet. El señor que nos dice qué tenemos que hacer para que no nos violen. // YES
@Russtophocles1 Girls don’t count as reply guys , call me sexist but I always love compliments from da gals. Even if they are horny posting😂😂 but if you’re mean I will cry // YES
Siempre he pensado esto de todos los abogados ... Jajaja Bueno mi cuñado NO es así, me sorprende porque es el estereotipo perfecto! Costeño, ebrio y abogado https://t.co/YH6AjDlElH // NO
Es algo relacionado con la identidad de g

In [ ]:
print(few_shot_samples_run1)

<s>[INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.
TWEET:@JackelineWB @JohnMAckerman @lopezobrador_ you are absurd, that is, Loret has the right to violate the privacy of civilians?? [/INST] 
 NO
</s> 

 <s>[INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.
TWEET:I wonder if the fact that there are fortunes generates a glass ceiling or something like that. There are more and more rich people, and anyone who is bothered by what they inherited should analyze whether they suffer from envy. https://t.co/oesBZ03sdh [/INST] 
 NO
</s> 

 <s>[INST] Is the following TWEET sexist, in any form, or does it describe situations in 

In [ ]:
print(few_shot_samples_run2)

[INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.
TWEET:@JackelineWB @JohnMAckerman @lopezobrador_ you are absurd, that is, Loret has the right to violate the privacy of civilians?? [/INST] 
 NO
 

 [INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discrimination occurs (i.e., it is sexist itself, describes a sexist situation or criticizes a sexist behaviour)? Reply only with YES or NO.
TWEET:I wonder if the fact that there are fortunes generates a glass ceiling or something like that. There are more and more rich people, and anyone who is bothered by what they inherited should analyze whether they suffer from envy. https://t.co/oesBZ03sdh [/INST] 
 NO
 

 [INST] Is the following TWEET sexist, in any form, or does it describe situations in which such discri

# Run!

In [ ]:
replies_list = ['YES','NO']
counter = 0

# List to store the results.
json_result_run1 = []

# simple JSON loading
with open(test_path, 'r') as istr:
    test_json = json.load(istr)
num_sample = len(test_json)
#print(num_sample)

#print(train_1_json[0])
for key, value in test_json.items():
  counter+=1
  tweet = test_json[key]['tweet']
  if test_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(test_json[key]['tweet'])

  current_sample = '[INST]\n'+tweet+'\n[/INST]'
  prompt = few_shot_samples_run1+current_sample

  #print(prompt)

  response = lcpp_llm(
        prompt=prompt,
        temperature= 0.2,
        logprobs=1,
        #max_tokens =1
      )

  #print(response)

  answer = str(response["choices"][0]["text"]).strip().upper()
  #print(answer)
  answer = answer[:3]
  #answer = answer.split()[0]
  # Sometime output contains a '.' remove it!
  #answer = answer.replace('.','')

  # If the predicted word is not in emotion list just replace with neutral.
  if answer not in replies_list:
        answer = 'NO'

  #current_sample += answer + " \n "

  print("GENERATED: "+ current_sample+'\n'+answer)

  current_element = {
        "id": test_json[key]['id_EXIST'],
        "value": answer,
        "test_case": "EXIST2024"
    }
  json_result_run1.append(current_element)

with open("task1_hard_badrock_1.json", "w") as json_file:
    json.dump(json_result_run1, json_file, indent=4)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =      11.31 ms /    16 runs   (    0.71 ms per token,  1415.05 tokens per second)
llama_print_timings: prompt eval time =   43991.62 ms /    31 tokens ( 1419.08 ms per token,     0.70 tokens per second)
llama_print_timings:        eval time =    1781.28 ms /    15 runs   (  118.75 ms per token,     8.42 tokens per second)
llama_print_timings:       total time =    2919.85 ms /    46 tokens


GENERATED: [INST]
@Eurogamer_es All gamergate from development to game forums, classic in the world of video games.
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =       8.94 ms /    16 runs   (    0.56 ms per token,  1789.51 tokens per second)
llama_print_timings: prompt eval time =     522.03 ms /    32 tokens (   16.31 ms per token,    61.30 tokens per second)
llama_print_timings:        eval time =    1484.48 ms /    15 runs   (   98.97 ms per token,    10.10 tokens per second)
llama_print_timings:       total time =    2144.45 ms /    47 tokens


GENERATED: [INST]
@ArCaNgEl__23 @Benzenazi Man, it's not comparable, look at Gamergate.
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =       9.12 ms /    16 runs   (    0.57 ms per token,  1754.19 tokens per second)
llama_print_timings: prompt eval time =     537.64 ms /    24 tokens (   22.40 ms per token,    44.64 tokens per second)
llama_print_timings:        eval time =    1515.20 ms /    15 runs   (  101.01 ms per token,     9.90 tokens per second)
llama_print_timings:       total time =    2172.92 ms /    39 tokens


...

GENERATED: [INST]
walkin back from the gym &amp; an older lady shouts to me from her car "you look like a whore!" - tell me why my natural reaction was to shout back "aw thanks!" 😂😂 https://t.co/PoEcMN6ka3
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =      11.51 ms /    16 runs   (    0.72 ms per token,  1390.34 tokens per second)
llama_print_timings: prompt eval time =     245.59 ms /    41 tokens (    5.99 ms per token,   166.95 tokens per second)
llama_print_timings:        eval time =    1892.42 ms /    15 runs   (  126.16 ms per token,     7.93 tokens per second)
llama_print_timings:       total time =    2359.22 ms /    56 tokens


GENERATED: [INST]
You look like a whore of Babylon bc that’s the only requirement for church drummer https://t.co/YbvZV43pMb
[/INST]
YES


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =       8.70 ms /    16 runs   (    0.54 ms per token,  1838.24 tokens per second)
llama_print_timings: prompt eval time =     805.52 ms /    30 tokens (   26.85 ms per token,    37.24 tokens per second)
llama_print_timings:        eval time =    1501.27 ms /    15 runs   (  100.08 ms per token,     9.99 tokens per second)
llama_print_timings:       total time =    2436.93 ms /    45 tokens


GENERATED: [INST]
@BellaLove823 @lolaoflucia You look like a whore. Stop projecting!
[/INST]
YES


In [ ]:
replies_list = ['YES','NO']
counter = 0

# List to store the results.
json_result_run2 = []

# simple JSON loading
with open(test_path, 'r') as istr:
    test_json = json.load(istr)
num_sample = len(test_json)
#print(num_sample)

#print(train_1_json[0])
for key, value in test_json.items():
  counter+=1
  tweet = test_json[key]['tweet']
  if test_json[key]['lang']=='es':
      tweet = GoogleTranslator(source='es', target='en').translate(test_json[key]['tweet'])

  current_sample = '[INST]\n'+tweet+'\n[/INST]'
  prompt = few_shot_samples_run2+current_sample

  #print(prompt)

  response = lcpp_llm(
        prompt=prompt,
        temperature= 0.2,
        logprobs=1,
        #max_tokens =1
      )

  #print(response)

  answer = str(response["choices"][0]["text"]).strip().upper()
  #print(answer)
  answer = answer[:3]
  #answer = answer.split()[0]
  # Sometime output contains a '.' remove it!
  #answer = answer.replace('.','')

  # If the predicted word is not in emotion list just replace with neutral.
  if answer not in replies_list:
        answer = 'NO'

  #current_sample += answer + " \n "

  print("GENERATED: "+ current_sample+'\n'+answer)

  current_element = {
        "id": test_json[key]['id_EXIST'],
        "value": answer,
        "test_case": "EXIST2024"
    }
  json_result_run2.append(current_element)

with open("task1_hard_badrock_2.json", "w") as json_file:
    json.dump(json_result_run2, json_file, indent=4)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =       9.76 ms /    16 runs   (    0.61 ms per token,  1640.02 tokens per second)
llama_print_timings: prompt eval time =    1825.37 ms /  1309 tokens (    1.39 ms per token,   717.12 tokens per second)
llama_print_timings:        eval time =    1682.95 ms /    15 runs   (  112.20 ms per token,     8.91 tokens per second)
llama_print_timings:       total time =    7063.25 ms /  1324 tokens


GENERATED: [INST]
@Eurogamer_es All gamergate from development to game forums, classic in the world of video games.
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =       9.93 ms /    16 runs   (    0.62 ms per token,  1611.93 tokens per second)
llama_print_timings: prompt eval time =     597.88 ms /    32 tokens (   18.68 ms per token,    53.52 tokens per second)
llama_print_timings:        eval time =    1530.68 ms /    15 runs   (  102.05 ms per token,     9.80 tokens per second)
llama_print_timings:       total time =    2315.81 ms /    47 tokens


GENERATED: [INST]
@ArCaNgEl__23 @Benzenazi Man, it's not comparable, look at Gamergate.
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =       8.86 ms /    16 runs   (    0.55 ms per token,  1805.26 tokens per second)
llama_print_timings: prompt eval time =     606.38 ms /    24 tokens (   25.27 ms per token,    39.58 tokens per second)
llama_print_timings:        eval time =    1479.91 ms /    15 runs   (   98.66 ms per token,    10.14 tokens per second)
llama_print_timings:       total time =    2206.88 ms /    39 tokens


...

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =       9.32 ms /    16 runs   (    0.58 ms per token,  1717.11 tokens per second)
llama_print_timings: prompt eval time =     275.73 ms /    70 tokens (    3.94 ms per token,   253.88 tokens per second)
llama_print_timings:        eval time =    1498.95 ms /    15 runs   (   99.93 ms per token,    10.01 tokens per second)
llama_print_timings:       total time =    1992.62 ms /    85 tokens


GENERATED: [INST]
Discover more at Bilibili Comics - Search for "The Forced Marriage" and add it to your favorites! #originalcharacter #imagecomics #Japanbookshttps://t.co/zsFOEtdfiW https://t.co/bR6g3XrGvX
[/INST]
NO


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1322.57 ms
llama_print_timings:      sample time =      10.64 ms /    16 runs   (    0.66 ms per token,  1504.18 tokens per second)
llama_print_timings: prompt eval time =     269.26 ms /    41 tokens (    6.57 ms per token,   152.27 tokens per second)
llama_print_timings:        eval time =    1885.72 ms /    15 runs   (  125.71 ms per token,     7.95 tokens per second)
llama_print_timings:       total time =    2411.80 ms /    56 tokens


GENERATED: [INST]
What deceptive weather, I came out in balls because it was hot and on the way back, it was a bit of a surprise for me. I caught pneumonia
[/INST]
NO


Llama.generate: prefix-match hit


In [ ]:
!zip -r exist2024_badrock.zip task1_hard_badrock_1.json task1_hard_badrock_2.json

In [ ]:
from google.colab import files
files.download('exist2024_badrock.zip')